# Machine Learning - Serverless Hosting

Machine Learning models can be hosted on server less functions. To host this model, we must consider the size of the packages and their security settings. 

In [ ]:
# download the file

!wget https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

2023-11-27 12:51:50.728457: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 12:51:51.454574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

def convert_keras_model(path, output_path):
    """
    Convert a keras model to TensorFlow Lite format and save it to a file.
    """
    
    model = load_model(path)
   
    # Convert the model to TensorFlow Lite format   
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    lite_model = converter.convert()

    # Save the converted model to a file
    with open(output_path, "wb") as f:
        f.write(lite_model)
        # display the byte size of the model
        print("Size of the model(MB): ", round(len(lite_model) / 1024 / 1024, 0))

## Question 1 -  convert this model from Keras to TF-Lite format. What is the size?

What's the size of the converted model?

- 21 Mb
- 43 Mb
- 80 Mb
- 164 Mb


In [3]:
# convert the model
convert_keras_model('./models/bees-wasps.h5', './models/bees-wasps.tflite')


INFO:tensorflow:Assets written to: /tmp/tmp1x84toli/assets


INFO:tensorflow:Assets written to: /tmp/tmp1x84toli/assets
2023-11-27 12:52:04.260632: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-27 12:52:04.260704: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-27 12:52:04.261411: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp1x84toli
2023-11-27 12:52:04.262389: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-11-27 12:52:04.262405: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp1x84toli
2023-11-27 12:52:04.264835: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-11-27 12:52:04.265460: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-27 12:52:04.468976: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

Size of the model(MB):  43.0


## Question 2 - What's the output index for this model?

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

- 3
- 7
- 13
- 24


In [5]:
# Image processing code

# pip install pillow 

from io import BytesIO
from urllib import request
from PIL import Image  

def download_image(url):
    """
    download the image from the url and return a stream
    """
    with request.urlopen(url) as resp:
        buffer = resp.read()
    
    stream = BytesIO(buffer)
    img = Image.open(stream)

    return img


def prepare_image(img, target_size):
    """
    Resize the image to target_size i.e. (150,150)
    """
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(img):
    """
    convert the image to a numpy array and normalize it
    """
    # convert to numpy array
    img = np.array(img)
    
    # convert to float32 to avoid overflow when multiplying by 255
    img = img.astype('float32')
    
    # normalize to the range 0-1
    img /= 255

    return img  

def load_lite_model(path):
    """
    Load the TensorFlow Lite model and allocate tensors.
    """
    # Load the TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()    

    return interpreter, input_details, output_details


In [8]:
interpreter, input_details, output_details = load_lite_model('./models/bees-wasps.tflite')


# Print details for input tensor(s)
print("Input details:", input_details[0]['index'])
# Print details for output tensor(s)
print("\nOutput details:", output_details[0]['index'])


Input details: 0

Output details: 13


## Question 3 -  what's the value in the first pixel, the R channel?

Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- 0.3450980
- 0.5450980
- 0.7450980
- 0.9450980


In [12]:
def download_and_preprocess_image(url, target_size):
    """
    Download the image from the url, resize it to target_size and return a numpy array
    """
    
    img_stream = download_image(url)
    img = prepare_image(img_stream, target_size)
    img_normalized = preprocess_image(img)
       
    return img_normalized

# download the image
image_url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

# convert the image to numpy array
img_normalized = download_and_preprocess_image(image_url, (150, 150))

# print the first pixel on the R channel (normalized)
print(img_normalized[0, 0, 0])
    

0.94509804


## Question 4 -  What's the output of the model?

Now let's apply this model to this image. What's the output of the model?

- 0.258
- 0.458
- 0.658
- 0.858

In [16]:
def img_inference(model_path, img_normalized):
    """
    Load the TensorFlow Lite model and run the inference.
    """

    # load the lite model and the input/output details
    interpreter, input_details, output_details = load_lite_model('./models/bees-wasps.tflite')

    # print the input shape and type
    print(input_details[0]['shape'])
    print(input_details[0]['dtype'])

    # print the output shape and type
    print(output_details[0]['shape'])
    print(output_details[0]['dtype'])

    # set the input tensor with the normalized image
    interpreter.set_tensor(input_details[0]['index'], [img_normalized])

    # run the inference
    interpreter.invoke()

    # get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # get the result value
    tensor_result = round(output_data[0][0],3)

    # print the output
    print('Tensor Output', tensor_result)

    return output_data[0].tolist()

In [17]:
# run the inference on a new image
model_url = './models/bees-wasps.tflite'
# run the inference
result = img_inference(model_url, img_normalized)

print(result)

[  1 150 150   3]
<class 'numpy.float32'>
[1 1]
<class 'numpy.float32'>
Tensor Output 0.659
[0.6592137217521667]


## Export the code and add the virtual environment

- Create the virtual environment
- Install the dependencies

```bash
pipenv shell
pipenv install pillow tflite_runtime

```
- Use tensorflow lite instead of Tensorflow

```bash
import tflite_runtime.interpreter as tflite
```


In [3]:
# save the notebook to a python file 
!jupyter nbconvert --to script homework.ipynb

# rename the homework.py file to bees-wasps.py
!mv homework.py bees_wasps.py

## Question 5 - What is the size of the base Docker image?

Download the base image agrigorev/zoomcamp-bees-wasps:v2. You can easily make it by using docker pull command.

So what's the size of this base image?

- 162 Mb
- 362 Mb
- 662 Mb
- 962 Mb

In [5]:
!docker images | grep bees-wasps

agrigorev/zoomcamp-bees-wasps   v2             b9f6c13de368   4 days ago      662MB


## Question 6 - What is the score of the image using the Docker instance?

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is bees-wasps-v2.tflite and it's in the current workdir in the image (see the Dockerfile above for the reference).

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

- 0.2453
- 0.4453
- 0.6453
- 0.8453

In [22]:
import requests
import json

# call the docker url with the image url localhost:8080

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'
img_url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

# call the docker url with the image url
response = requests.post(url, data=json.dumps({'url': img_url})).json()
print('docker container result',response)



docker container result [0.4453350603580475]
